Queremos averiguar cuales son las tendencias sexuales de los usuarios de Tinder y clasificarlos usando KMeans.
Para eso vamos a escoger los features que solo tengan las palabra "sex" y escoger los atributos que veamos más 
conveniente, por otro lado atributos como timestap es irrelevante asi que lo quitamos ya.

In [258]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import re
import seaborn as sns

In [259]:
df = pd.read_csv("/Users/fujiemei/Desktop/tinder.csv")

Creemos que el género,se puede quitar porque la orientación, la edad también no la encontramos relevante y es
curioso ver que hay hay edades <18 cuando la aplicación es para gente adulta, suponemos que el usuario a la hora 
de entrar datos es haya equivocado. La educación tampoco la encontramos relevante también como la convivencia por lo
que los podemos eliminar también.Tampoco hay ningún valor nulo por lo que tendremos que buscar valores fuera de lo
común de las columnas.

In [260]:
df = df.drop(columns = ["Marca temporal"])
df

,Edad,Género,Orientación sexual,Educación,Religión,Convivencia,Estado civil,¿Qué tan importante es su religión en su vida diaria?,Edad de la primera relación sexual,Usa drogas durante el sexo,...,Necesidad de dejarme llevar sexualmente con alguien,"Necesidad de alguien que sea ""genial en la cama""",Necesidad de sexo con muchas parejas,Necesidad de una pareja que me quiera,Necesidad de alguien a quien amar,Necesidad de un compañero en la vida,Necesidad de confianza total en las personas con las que tengo intimidad,No necesito a nadie especial en mi vida,Necesidad de algunos muy buenos amigos,Necesidad de alguien que duerma a mi lado
0,15,Mujer,Heterosexual,Superior,Ateo / agnóstico,Con padres o tutores,Soltero pero saliendo casualmente,Nada importante,10,No,...,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo
1,28,Mujer,Heterosexual,Superior,Ateo / agnóstico,Con padres o tutores,Soltero pero saliendo casualmente,Nada importante,22,No,...,En desacuerdo,De acuerdo,En desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo
2,29,Varon,Heterosexual,Superior,Católico,Solo o con compañero de habitación,Soltero pero saliendo casualmente,Normal,18,No,...,De acuerdo,En desacuerdo,En desacuerdo,De acuerdo,De acuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo
3,28,Varon,Heterosexual,Superior,Ateo / agnóstico,Solo o con compañero de habitación,Soltero pero saliendo casualmente,No tan importante,19,No,...,De acuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo
4,25,Mujer,Heterosexual,Superior,"Otros (Islamismo, Budismo, Hinduismo, Judaismo)",Con padres o tutores,Soltero pero saliendo casualmente,Importante,17,No,...,Totalmente en desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,26,Mujer,Heterosexual,Superior,Católico,Solo o con compañero de habitación,En una relación seria,Importante,18,No,...,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo
181,24,Mujer,Heterosexual,Superior,Católico,Con padres o tutores,Soltero pero saliendo casualmente,Muy importante,19,No,...,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,De acuerdo,En desacuerdo,En desacuerdo
182,19,Mujer,Heterosexual,Superior,Católico,Con padres o tutores,Soltero pero saliendo casualmente,Muy importante,19,No,...,En desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,De acuerdo,En desacuerdo,Totalmente de acuerdo,En desacuerdo
183,22,Mujer,Heterosexual,Superior,Católico,Con padres o tutores,Soltero pero saliendo casualmente,No tan importante,18,No,...,En desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,En desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo


In [261]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 68 columns):
 #   Column                                                                                                                  Non-Null Count  Dtype 
---  ------                                                                                                                  --------------  ----- 
 0   Edad                                                                                                                    185 non-null    int64 
 1   Género                                                                                                                  185 non-null    object
 2   Orientación sexual                                                                                                      185 non-null    object
 3   Educación                                                                                                               185 non-null    object
 4   Re

In [262]:
df = df.drop(columns = ["Edad","Género","Educación","Convivencia"])
df

,Orientación sexual,Religión,Estado civil,¿Qué tan importante es su religión en su vida diaria?,Edad de la primera relación sexual,Usa drogas durante el sexo,Usa bebidas alcohólicas durante el sexo,¿Algún antecedente de infecciones de transmisión sexual?,¿Actualmente padece de alguna infección de transmisión sexual?,¿Alguna vez estuvo embarazada o dejó embarazada a otra persona?,...,Necesidad de dejarme llevar sexualmente con alguien,"Necesidad de alguien que sea ""genial en la cama""",Necesidad de sexo con muchas parejas,Necesidad de una pareja que me quiera,Necesidad de alguien a quien amar,Necesidad de un compañero en la vida,Necesidad de confianza total en las personas con las que tengo intimidad,No necesito a nadie especial en mi vida,Necesidad de algunos muy buenos amigos,Necesidad de alguien que duerma a mi lado
0,Heterosexual,Ateo / agnóstico,Soltero pero saliendo casualmente,Nada importante,10,No,No,No,No,No,...,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo
1,Heterosexual,Ateo / agnóstico,Soltero pero saliendo casualmente,Nada importante,22,No,No,No,No,No,...,En desacuerdo,De acuerdo,En desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo
2,Heterosexual,Católico,Soltero pero saliendo casualmente,Normal,18,No,No,No,No,No,...,De acuerdo,En desacuerdo,En desacuerdo,De acuerdo,De acuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo
3,Heterosexual,Ateo / agnóstico,Soltero pero saliendo casualmente,No tan importante,19,No,Sí,No,No,No,...,De acuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo
4,Heterosexual,"Otros (Islamismo, Budismo, Hinduismo, Judaismo)",Soltero pero saliendo casualmente,Importante,17,No,No,Sí,No,Sí,...,Totalmente en desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,Heterosexual,Católico,En una relación seria,Importante,18,No,Sí,Sí,No,No,...,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo
181,Heterosexual,Católico,Soltero pero saliendo casualmente,Muy importante,19,No,No,No,No,No,...,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,De acuerdo,En desacuerdo,En desacuerdo
182,Heterosexual,Católico,Soltero pero saliendo casualmente,Muy importante,19,No,No,No,No,No,...,En desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,De acuerdo,En desacuerdo,Totalmente de acuerdo,En desacuerdo
183,Heterosexual,Católico,Soltero pero saliendo casualmente,No tan importante,18,No,Sí,No,No,No,...,En desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,En desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo


In [263]:
x = df.columns
i = 0
x = x.to_numpy().tolist()
sex = []
for word in x:
    if re.search("sexo",word) !=None:
        sex.append(word)

sex    

['Usa drogas durante el sexo',
 'Usa bebidas alcohólicas durante el sexo',
 '¿Cuántas de las personas que conociste en Tinder, te comentaron haber  usaron drogas y/o alcohol durante el sexo?',
 '¿Mis deseos de tener sexo interrumpen mi vida diaria?',
 '¿Me encuentro pensando en sexo en el trabajo?',
 '¿Pienso en el sexo más de lo que me gustaría?',
 '¿Es difícil para mí encontrar parejas sexuales que deseen tener sexo tanto como yo quiero?',
 'El sexo casual es aceptable',
 'Me gustaría tener sexo con muchas parejas',
 'El sexo como un simple intercambio de favores está bien si ambas personas están de acuerdo',
 'El mejor sexo es sin ataduras',
 'Es posible disfrutar del sexo con una persona y no gustar mucho esa persona',
 'Está bien que el sexo sea solo una buena liberación física',
 'El sexo es la forma más cercana de comunicación entre dos personas',
 'En el mejor de los casos, el sexo parece ser la fusión de dos almas',
 'El sexo es una parte muy importante de la vida',
 'El sexo 

In [264]:
useless = ['Usa drogas durante el sexo',
 'Usa bebidas alcohólicas durante el sexo',
 '¿Cuántas de las personas que conociste en Tinder, te comentaron haber  usaron drogas y/o alcohol durante el sexo?',]
for word in useless:
    sex.remove(word)
sex

['¿Mis deseos de tener sexo interrumpen mi vida diaria?',
 '¿Me encuentro pensando en sexo en el trabajo?',
 '¿Pienso en el sexo más de lo que me gustaría?',
 '¿Es difícil para mí encontrar parejas sexuales que deseen tener sexo tanto como yo quiero?',
 'El sexo casual es aceptable',
 'Me gustaría tener sexo con muchas parejas',
 'El sexo como un simple intercambio de favores está bien si ambas personas están de acuerdo',
 'El mejor sexo es sin ataduras',
 'Es posible disfrutar del sexo con una persona y no gustar mucho esa persona',
 'Está bien que el sexo sea solo una buena liberación física',
 'El sexo es la forma más cercana de comunicación entre dos personas',
 'En el mejor de los casos, el sexo parece ser la fusión de dos almas',
 'El sexo es una parte muy importante de la vida',
 'El sexo suele ser una experiencia intensa, casi abrumadora',
 'El sexo es mejor cuando te dejas llevar y te concentras en tu propio placer',
 'El sexo es principalmente obtener placer de otra persona',

In [265]:
unique = df['El mejor sexo es sin ataduras'].unique()
unique

array(['En desacuerdo', 'Ni de acuerdo ni en desacuerdo',
       'Totalmente en desacuerdo', 'De acuerdo', 'Totalmente de acuerdo'],
      dtype=object)

In [266]:
def valoresCorrectos(unique,word):
    for words in unique:
        if words==word:return True
    return False

def allOk(sex,unique):
    badCol = []
    for columns in sex:
        x = df[columns]
        for w in x:
            if (not valoresCorrectos(unique,w)): 
                badCol.append(columns)
                break
            
    return badCol
badCol = allOk(sex,unique)
badCol

['¿Mis deseos de tener sexo interrumpen mi vida diaria?',
 '¿Me encuentro pensando en sexo en el trabajo?',
 '¿Pienso en el sexo más de lo que me gustaría?',
 '¿Es difícil para mí encontrar parejas sexuales que deseen tener sexo tanto como yo quiero?',
 'El objetivo principal del sexo es divertirse']

In [267]:
unique2 = df['¿Pienso en el sexo más de lo que me gustaría?'].unique()
badCol2 = allOk(badCol,unique2)
badCol2

['El objetivo principal del sexo es divertirse']

In [268]:
print(sorted(unique))
print(sorted(df["El objetivo principal del sexo es divertirse"].unique()))

['De acuerdo', 'En desacuerdo', 'Ni de acuerdo ni en desacuerdo', 'Totalmente de acuerdo', 'Totalmente en desacuerdo']
['De acuerdo', 'En desacuerdo', 'Ni de acuerdo ni en desacuerdo', 'Totalmente de acuerdo', 'Totalmente en descuerdo']


In [271]:
df["El objetivo principal del sexo es divertirse"] = df["El objetivo principal del sexo es divertirse"].replace(
                                                        "Totalmente en descuerdo",'Totalmente en desacuerdo')
df["El objetivo principal del sexo es divertirse"].unique()

array(['En desacuerdo', 'Ni de acuerdo ni en desacuerdo', 'De acuerdo',
       'Totalmente de acuerdo', 'Totalmente en desacuerdo'], dtype=object)

In [154]:
df["Religión"] = df['Religión'].replace(["Católico", "Otros (Islamismo, Budismo, Hinduismo, Judaismo)"]
                                          ,"Creyente")


,Orientación sexual,Religión,Estado civil,¿Qué tan importante es su religión en su vida diaria?,Edad de la primera relación sexual,Usa drogas durante el sexo,Usa bebidas alcohólicas durante el sexo,¿Algún antecedente de infecciones de transmisión sexual?,¿Actualmente padece de alguna infección de transmisión sexual?,¿Alguna vez estuvo embarazada o dejó embarazada a otra persona?,...,Necesidad de dejarme llevar sexualmente con alguien,"Necesidad de alguien que sea ""genial en la cama""",Necesidad de sexo con muchas parejas,Necesidad de una pareja que me quiera,Necesidad de alguien a quien amar,Necesidad de un compañero en la vida,Necesidad de confianza total en las personas con las que tengo intimidad,No necesito a nadie especial en mi vida,Necesidad de algunos muy buenos amigos,Necesidad de alguien que duerma a mi lado
0,Heterosexual,Ateo / agnóstico,Soltero pero saliendo casualmente,Nada importante,10,No,No,No,No,No,...,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo
1,Heterosexual,Ateo / agnóstico,Soltero pero saliendo casualmente,Nada importante,22,No,No,No,No,No,...,En desacuerdo,De acuerdo,En desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo
2,Heterosexual,Creyente,Soltero pero saliendo casualmente,Normal,18,No,No,No,No,No,...,De acuerdo,En desacuerdo,En desacuerdo,De acuerdo,De acuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo
3,Heterosexual,Ateo / agnóstico,Soltero pero saliendo casualmente,No tan importante,19,No,Sí,No,No,No,...,De acuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo
4,Heterosexual,Creyente,Soltero pero saliendo casualmente,Importante,17,No,No,Sí,No,Sí,...,Totalmente en desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,Heterosexual,Creyente,En una relación seria,Importante,18,No,Sí,Sí,No,No,...,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo
181,Heterosexual,Creyente,Soltero pero saliendo casualmente,Muy importante,19,No,No,No,No,No,...,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,De acuerdo,En desacuerdo,En desacuerdo
182,Heterosexual,Creyente,Soltero pero saliendo casualmente,Muy importante,19,No,No,No,No,No,...,En desacuerdo,Ni de acuerdo ni en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,De acuerdo,En desacuerdo,Totalmente de acuerdo,En desacuerdo
183,Heterosexual,Creyente,Soltero pero saliendo casualmente,No tan importante,18,No,Sí,No,No,No,...,En desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,En desacuerdo,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo


In [157]:
df = pd.get_dummies(df,columns = ["Orientación sexual"],drop_first = True)
df = pd.get_dummies(df,columns = ["El sexo es la forma más cercana de comunicación entre dos personas",
                                  "Un encuentro sexual entre dos personas profundamente enamoradas es la máxima interacción humana",
                                 "El sexo es una parte muy importante de la vida",
                                 "El sexo es una parte muy importante de la vida",
                                 "El sexo suele ser una experiencia intensa, casi abrumadora",
                                 "Necesidad de una pareja que me quiera",
                                 "Necesidad de confianza total en las personas con las que tengo intimidad",
                                 "No necesito a nadie especial en mi vida"])


In [158]:
df

,Edad,Educación,Religión,Convivencia,Estado civil,"En el mejor de los casos, el sexo parece ser la fusión de dos almas",El sexo es mejor cuando te dejas llevar y te concentras en tu propio placer,El sexo es principalmente obtener placer de otra persona,El objetivo principal del sexo es divertirse,El sexo es principalmente físico,...,Necesidad de confianza total en las personas con las que tengo intimidad_De acuerdo,Necesidad de confianza total en las personas con las que tengo intimidad_En desacuerdo,Necesidad de confianza total en las personas con las que tengo intimidad_Ni de acuerdo ni en desacuerdo,Necesidad de confianza total en las personas con las que tengo intimidad_Totalmente de acuerdo,Necesidad de confianza total en las personas con las que tengo intimidad_Totalmente en desacuerdo,No necesito a nadie especial en mi vida_De acuerdo,No necesito a nadie especial en mi vida_En desacuerdo,No necesito a nadie especial en mi vida_Ni de acuerdo ni en desacuerdo,No necesito a nadie especial en mi vida_Totalmente de acuerdo,No necesito a nadie especial en mi vida_Totalmente en desacuerdo
0,15,Superior,Ateo / agnóstico,Con padres o tutores,Soltero pero saliendo casualmente,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,...,0,1,0,0,0,0,1,0,0,0
1,28,Superior,Ateo / agnóstico,Con padres o tutores,Soltero pero saliendo casualmente,Ni de acuerdo ni en desacuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,...,1,0,0,0,0,0,0,1,0,0
2,29,Superior,Creyente,Solo o con compañero de habitación,Soltero pero saliendo casualmente,De acuerdo,De acuerdo,En desacuerdo,En desacuerdo,En desacuerdo,...,0,0,1,0,0,0,1,0,0,0
3,28,Superior,Ateo / agnóstico,Solo o con compañero de habitación,Soltero pero saliendo casualmente,De acuerdo,Totalmente de acuerdo,En desacuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,...,0,0,0,1,0,0,0,0,0,1
4,25,Superior,Creyente,Con padres o tutores,Soltero pero saliendo casualmente,Ni de acuerdo ni en desacuerdo,De acuerdo,De acuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,26,Superior,Creyente,Solo o con compañero de habitación,En una relación seria,Ni de acuerdo ni en desacuerdo,De acuerdo,En desacuerdo,De acuerdo,De acuerdo,...,0,0,0,1,0,0,0,0,0,1
181,24,Superior,Creyente,Con padres o tutores,Soltero pero saliendo casualmente,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,En desacuerdo,...,0,1,0,0,0,1,0,0,0,0
182,19,Superior,Creyente,Con padres o tutores,Soltero pero saliendo casualmente,De acuerdo,Ni de acuerdo ni en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,...,1,0,0,0,0,0,1,0,0,0
183,22,Superior,Creyente,Con padres o tutores,Soltero pero saliendo casualmente,Ni de acuerdo ni en desacuerdo,De acuerdo,Ni de acuerdo ni en desacuerdo,En desacuerdo,Ni de acuerdo ni en desacuerdo,...,0,1,0,0,0,0,0,1,0,0
